In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ret = []
path = './imgs/unknown'
for i in range(1, 9):
    o_path = os.path.join(path, 'o{}'.format(i) + '.png')
    r_path = os.path.join(path, 'r{}'.format(i) + '.png')
    o_img = cv2.cvtColor(cv2.imread(o_path), cv2.COLOR_BGR2GRAY)
    r_img = cv2.cvtColor(cv2.imread(r_path), cv2.COLOR_BGR2GRAY)
    res = np.zeros(o_img.shape)
    res[o_img > r_img] = 1
    ret.append(cv2.resize(res, dsize=(res.shape[1] // 2, res.shape[0] // 2)))


In [3]:
ret_none = []
path = './imgs/unknown'
for i in range(1, 9):
    o_path = os.path.join(path, 'no{}'.format(i) + '.png')
    r_path = os.path.join(path, 'nr{}'.format(i) + '.png')
    o_img = cv2.cvtColor(cv2.imread(o_path), cv2.COLOR_BGR2GRAY)
    r_img = cv2.cvtColor(cv2.imread(r_path), cv2.COLOR_BGR2GRAY)
    res = np.zeros(o_img.shape)
    res[o_img > r_img] = 1
    ret_none.append(cv2.resize(res, dsize=(res.shape[1] // 2, res.shape[0] // 2)))


In [4]:
cv2.imshow('result1', ret[5])
cv2.imshow('resilt2', ret_none[5])
cv2.waitKey()
cv2.destroyAllWindows()

In [5]:
ret = np.zeros(o_img.shape, dtype=np.uint8)
path = './imgs/unknown'
for i in range(1, 9):
    o_path = os.path.join(path, 'o{}'.format(i) + '.png')
    r_path = os.path.join(path, 'r{}'.format(i) + '.png')
    o_img = cv2.cvtColor(cv2.imread(o_path), cv2.COLOR_BGR2GRAY)
    r_img = cv2.cvtColor(cv2.imread(r_path), cv2.COLOR_BGR2GRAY)
    ret *= 2
    ret[o_img > r_img] += 1


In [6]:
ret = cv2.resize(ret, dsize=(ret.shape[1] // 2, ret.shape[0] // 2))

In [7]:
ret_none = np.zeros(o_img.shape, dtype=np.uint8)
path = './imgs/unknown'
for i in range(1, 9):
    o_path = os.path.join(path, 'no{}'.format(i) + '.png')
    r_path = os.path.join(path, 'nr{}'.format(i) + '.png')
    o_img = cv2.cvtColor(cv2.imread(o_path), cv2.COLOR_BGR2GRAY)
    r_img = cv2.cvtColor(cv2.imread(r_path), cv2.COLOR_BGR2GRAY)
    ret_none *= 2
    ret_none[o_img > r_img] += 1


In [8]:
ret_none = cv2.resize(ret_none, dsize=(ret_none.shape[1] // 2, ret_none.shape[0] // 2))

In [9]:
cv2.imshow('1', ret)
cv2.imshow('2', ret_none)
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
imgL = ret
imgR = ret_none

In [11]:
ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''

def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [17]:
window_size = 5
min_disp = 4
num_disp = 128 - min_disp
# min_disp = 4
# num_disp = 20-min_disp
stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = 4,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2,
    disp12MaxDiff = 4,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 100,
    # mode=cv22.STEREO_SGBM_MODE_SGBM_3WAY
)

print('computing disparity...')
disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

print('generating 3d point cloud...',)
h, w = imgL.shape[:2]
# f = 0.8*w                          # guess for focal length
f = 394.59508339
Q = np.float32([[1, 0,  0, -0.5*w],
                [0, -1, 0, 0.5*h], # turn points 180 deg around x-axis,
                [0, 0,  0, -f], # so that y-axis looks up
                [0, 0,  1, 0]])
# Q = np.float32([[1, 0, 0, 0],
#             [0, -1, 0, 0], # turn points 180 deg around x-axis,
#             [0, 0, 0, -f], # so that y-axis looks up
#             [0, 0, 1, 0]])
# Q = np.float32([[1, 0, 0, 0],
#                 [0, 1, 0, 0],
#                 [0, 0, 1, 0],
#                 [0, 0, 0, 1]])
points = cv2.reprojectImageTo3D(disp, Q)
colors = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)
mask = disp > disp.min()
out_points = points[mask]
out_colors = colors[mask]
out_fn = 'out.ply'
write_ply(out_fn, out_points, out_colors)
print('%s saved' % out_fn)

cv2.imshow('left', imgL)
cv2.imshow('disparity', (disp-min_disp)/num_disp)
cv2.waitKey()
cv2.destroyAllWindows()

computing disparity...
generating 3d point cloud...
out.ply saved


In [61]:
cv2.imwrite('l.png', imgL)
cv2.imwrite('r.png', imgR)

True

In [96]:
imgl = imgL[100:550, 320:800]
imgr = imgR[200:500, 320:800]
# img = np.abs(imgL[200:500, 320:800] - imgR[200:500, 320:800])
cv2.imshow('left', imgl)
cv2.waitKey()
cv2.destroyAllWindows()

In [97]:
# a = np.where(img == )
# np.max(img)
# np.mean(a[1])

In [98]:
res = np.zeros(imgl.shape)
for i in range(256):
    a = np.where(imgl == i)
    b = np.where(imgr == i)
    if len(a[1]) == 0 or len(b[1]) == 0:
        continue
    # print(a[1])
    # break
    # if len(a) == 0:
        # continue
    # print(np.mean(a[1]) - np.mean(b[1]))
    res[imgl == i] = np.abs(np.mean(a[1]) - np.mean(b[1]))

In [99]:
# print(res)
res = res*10
res = np.floor(res)
res = res.astype(np.uint8)
print(np.max(res))
print(res.dtype)

247
uint8


In [100]:
cv2.imshow('left', res)
cv2.waitKey()
cv2.destroyAllWindows()

In [144]:
def meanBlur(src, ksize):
    '''
    INPUT:
    src: input image
    ksize: kernel size, tuple or intger
    
    OUTPUT:
    dst: return image 
    '''

    print("Running medianBlur...")
    print("It may takes about 10~20 seconds to finish it...")

    # 得到图片的相关信息
    width, height = src.shape
    rst = np.zeros((width, height), dtype=np.int32)
    padding_size = ksize // 2
    img = np.pad(src, ((padding_size, padding_size), (padding_size, padding_size)), mode='edge')
    
    # 取中位数作为点的值
    for i in range(width):
        for j in range(height):
            rst[i, j] = np.min(img[i:i+ksize, j:j+ksize],  axis=(0, 1))

    return rst

In [145]:
res_ = meanBlur(res, 3)

Running medianBlur...
It may takes about 10~20 seconds to finish it...


In [146]:
t = np.hstack((res_, imgl))
cv2.imwrite('3.png', t)

True

In [147]:
print(res.shape)
points = [[i, j, res_[i, j]] for i in range(res.shape[0]) for j in range(res.shape[1])]
print(points[0])
points = np.array(points)

(450, 480)
[0, 0, 11]


In [148]:
ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
end_header
'''

def write_ply(fn, verts):
    verts = verts.reshape(-1, 3)
    # colors = colors.reshape(-1, 3)
    # verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f')

In [149]:
write_ply('out.ply', points)